### Import/Load Data

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.learn.python.learn.datasets import base


# Load dataset
training_data = "iris_training.csv"
testing_data = "iris_test.csv"
training_set = base.load_csv_with_header(filename=training_data,
                                         features_dtype=np.float32, 
                                         target_dtype=np.int)
test_set = base.load_csv_with_header(filename=testing_data,
                                     features_dtype=np.float32,
                                     target_dtype=np.int)

In [2]:
feature_name = "iris_features"
feature_columns = [tf.feature_column.numeric_column(feature_name, shape=[4])]

### Input Function

In [3]:
def input_fn(data):
    def _fn():
        features = {feature_name: tf.constant(data.data)}
        label = tf.constant(data.target)
        return features, label
    return _fn

### Training w/ Linear Estimator

In [4]:
classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=3,
    model_dir="tmp/iris")

classifier.train(input_fn=input_fn(training_set),
               steps=100)

accuracy_score = classifier.evaluate(input_fn=input_fn(test_set), 
                                     steps=50)["accuracy"]
print('\n\n Accuracy: {0:f}'.format(accuracy_score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a8be5f8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from tmp/iris/model.ckpt-100
INFO:tensorflow:Saving checkpoints for 101 into tmp/iris/model.ckpt.
INFO:tensorflow:loss = 37.1391, step = 101
INFO:tensorflow:Saving checkpoints for 200 into tmp/iris/model.ckpt.
INFO:tensorflow:Loss for final step: 27.9227.
INFO:tensorflow:Starting evaluation at 2018-01-14-23:41:26
INFO:tenso

### Training w/ DNN Estimator

In [5]:
nn_classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[64, 12, 4],
    activation_fn=tf.nn.relu,
    dropout=0.4,
    n_classes=4,
    optimizer="Adam",    
    model_dir="tmp/irisnn")

nn_classifier.train(input_fn=input_fn(training_set),
               steps=1000)

accuracy_score = nn_classifier.evaluate(input_fn=input_fn(test_set), 
                                     steps=1000)["accuracy"]
print('\n\n Accuracy: {0:f}'.format(accuracy_score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp/irisnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11af652b0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from tmp/irisnn/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into tmp/irisnn/model.ckpt.
INFO:tensorflow:loss = 46.3033, step = 1001
INFO:tensorflow:global_step/sec: 343.628
INFO:tensorflow:loss = 54.4822, step = 1101 (0.289 sec)
INFO:tensorflow:global_step/sec: 723.931
INFO:tensorflow:loss = 46.1975, step 

### Serving function

In [6]:
feature_spec = {'iris_features': 
                tf.FixedLenFeature(shape=[4], dtype=np.float32)}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

### Export Model for Prediction

In [7]:
feature_spec = {'iris_features': tf.FixedLenFeature(shape=[4], dtype=np.float32)}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

classifier.export_savedmodel(export_dir_base='tmp/irisnn/export', 
                            serving_input_receiver_fn=serving_fn)

INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Restoring parameters from tmp/iris/model.ckpt-200
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"tmp/irisnn/export/temp-b'1515973347'/saved_model.pb"


b'tmp/irisnn/export/1515973347'

In [12]:
new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
    
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print("New Samples, Class Predictions: {}\n".format(predicted_classes))

ValueError: Feature iris_features is not in features dictionary.